# Lab 4b: Database Creation, Population and Relational Algebra 

** Introduction **

Since Panda's dataframes are essentially tables and Panda's provides easy to use [SQL-like methods](https://pandas.pydata.org/pandas-docs/stable/comparison_with_sql.html) for manipulating and querying multiple tables, why do we still need databases? 

Almost all transactional data (that changes over time, and particularly data with records that are updated over time) is stored in databases as opposed to flat files. The reason for this is that databases typically offer persistence, scalability, support for concurrent users and transactions.  For enterprise computing, and other data critical tasks, flat files and ad hoc in-memory schemes simply can't compete with [ACID](https://en.wikipedia.org/wiki/ACID).  There has been thousands of years worth of development effort put into safely and efficiently handling very large databases led by the relational database community.  More recently [other forms of databases](https://en.wikipedia.org/wiki/NoSQL) (such as columnar databases, e.g., [MongoDB](https://www.mongodb.com/nosql-explained)) have also been working on handling this same problems.

As a data scientist in a corporate environment you will typically be expected to be able to form SQL queries to extract data relevant for your research from whatever databases are in use.  Since those databases are designed to enforce various business constraints (e.g., Normal form) they can be quite cumbersome query.  

In order to simply the querying process, the star schema introduced in the last lab, in lecture, and also the focus of this lab is often employed by data scientists for data warehouse design.  Here's a nice short [video](https://www.youtube.com/watch?v=q77B-G8CA24) about Star Schemas versus Normalized Schema Database systems, and how flattening (which often includes partial de-normalization) to a Star Schema allows for much easier database querying.


**Objective **

In this lab, you will use SQL statements to create a Star Schema database and then
use Python and more SQL to load it with appropriately preprocessed data after.  While doing this you will be become familiar with [SQLite databases](https://www.sqlite.org/about.html) and with using Python's [sqlite3](https://docs.python.org/3/library/sqlite3.html) sqlite interface package.

You will also use relational algebra expressions to define some queries.  


** Tasks **
1. Create a database based on a Star Schema
1. Populate the database using flat files
2. Write Relational Algebra expressions

### Using SQL

Using SQL directly and SQL with Python is very easy. Below are two examples of creating a database table, inserting data into the table, and querying data from the table.  The first should be run inside of `sqlite3` which you can start at the command line, as follows:
```
Zeus:lab4b Dan$ sqlite3 food.db
SQLite version 3.13.0 2016-05-18 10:57:30
Enter ".help" for usage hints.
sqlite> 
```
After you have a `sqlite>` prompt displayed in a Terminal window, please work through the example below using sequential copy and paste operations to the `sqlite>` prompt.

```
-- Below are some examples of using the SQLite via its interpreter.

-- delete a table if it is present
DROP TABLE IF EXISTS food;

-- create a table called food
CREATE TABLE food (
            id INTEGER,
            name text,
            amount float,
            unit text,
            PRIMARY KEY(id),
            UNIQUE (name));

-- insert some individual records into the food table
INSERT INTO food (name, amount, unit) VALUES ('apple', 3, 'whole');
INSERT INTO food (name, amount, unit) VALUES ('milk', 64, 'oz' );

-- insert multiple items into the food table
INSERT INTO food (name, amount, unit) VALUES ('chicken', 4, 'whole' ), 
                                             ('beer',16,'oz');                                              
-- retrieve information of interest
-- This query returns the first 3 sorted food names and entry ids 
SELECT name, id FROM food ORDER BY name ASC LIMIT 3;


-- show the current database schema (just shows CREATE statements)
.schema 

-- Exit the SQLite intrepreter
.exit
```

### Using Python & SQL
Below is the above example, but coded in Python using the [sqlite3](https://docs.python.org/3/library/sqlite3.html) package interface for sqlite.

##### Open a Connection:
The first thing you'll need to do when interacting with a SQLite database  via a computer program (and generally any SQL database) is to open up a connection (see below). If the database doesn't exist, the SQLite3 module will create one for you when you first request a connection to it.

In [ ]:
import sqlite3
conn = sqlite3.connect('food.db')

##### Create a Cursor Object:
You have now opened a connection with the database. Next, let's create a cursor object for handling SQL queries. A cursor is essentially a small work area in memory where one formulates queries before sending them to the database for execution, after execution, this same area holds query results.  The purpose of a cursor is to allow a program to work with HUGE query results row by row.  It gets its name because it acts a pointer (a cursor!) into query results.

In [ ]:
# create a cursor for the current connection
c = conn.cursor()

And here's the example recoded in Python.  ** The only thing missing is the .schema command, which isn't readily available via the python sqlite3 interface. **

In [ ]:
## Below are some examples of using SQLite via Python.
# Python specific setup:
import sqlite3
# connect to database, creates database if not already present
conn = sqlite3.connect('food.db')  # database is stored in file 'example.db'
# create cursor 
c = conn.cursor() # cursor are used to interact with a db

# Begin Example
# delete a table if it is present
c.execute('DROP TABLE IF EXISTS food')

# create a table called food
c.execute('''CREATE TABLE food (
            id INTEGER,
            name text,
            amount float,
            unit text,
            PRIMARY KEY(id),
            UNIQUE (name))''')
conn.commit()

# insert some individual records into the food table
try:
    c.execute('''INSERT INTO food (name, amount, unit) VALUES (?, ?, ?)''', ('apple', 3, 'whole'))
    c.execute('''INSERT INTO food (name, amount, unit) VALUES (?, ?, ?)''', ('milk', 64, 'oz' ))
except sqlite3.Error as e: # Executes code below when a sqlite3 error occurs  
    print("food, insert error:", e.args[0])

conn.commit()

# insert multiple recordds
try:
    c.executemany('''INSERT INTO food (name, amount, unit) VALUES (?, ?, ?)''', [('chicken', 4, 'whole' ), 
                                                                                 ('beer',128,'oz'),
                                                                                 ('beer',16,'oz')])
except sqlite3.Error as e:  # Executes code below when a sqlite3 error occurs  
    print("food, insert error:", e.args[0])

conn.commit()

# retrieve information of interest
# This query returns the first 3 sorted food names and entry ids 
c.execute('''SELECT name, id FROM food ORDER BY name ASC LIMIT 3''')

# display query results
for r in c:
    print(r)

# shut the connection to the database
conn.close()

** Task 0: ** Figure out why there was an insert error.  How can it be eliminated?

your solution here

### Create your own database

In **Lab 4a** you designed a star schema to represent class enrollments. Today you will work with python to create a set of tables appropriate for the following course-enrollment schema:

<img src="student_star_schema.jpg", width=500>

In the above schema each row the fact table represents a course that a student has taken and the grade that they received.

**Hints:** 
- It is often easier to debug sql queries and commands using the SQL interpreter

In Python,
- to execute queries, simply run the `cursor.execute()` with your query as the argument.
- Remember to define `PRIMARY KEY`s and `FOREIGN KEY`s in your table, refer to [here](http://www.sqlitetutorial.net/sqlite-create-table/)
- Remember to use `conn.commit()` to commit any changes you've made to the database
- If you get a database locked message try using the Jupyter Notebook Kernel menu item to restart the kernel 

** TASK 1: Complete the following `create_database()` function **
Note: In the code below we include a line that drops pre-existing database tables related to your schema, doing this will allow you run your create_database function repeatedly.

In [ ]:
import sqlite3
conn = sqlite3.connect('student.db')

def create_database(conn):
    '''Creates tables appropriate for the Star Schema developed in Lab 4a'''
    c = conn.cursor()
    c.execute('''DROP TABLE IF EXISTS students''')
    c.execute('''DROP TABLE IF EXISTS courses''')
    c.execute('''DROP TABLE IF EXISTS fact_table''')
    
    # Create database with tables.
    
    c.execute('''
            CREATE TABLE students(
                id INTEGER,
                student text,
                age int,
                degree text,
                PRIMARY KEY(id),
                UNIQUE (student))''')
    
    
    # Create courses table 
    # YOUR CODE HERE
    
    # Uncomment and complete the fact_table
#    c.execute('''
#            CREATE TABLE fact_table(
#                student_id INTEGER,
#
#                FOREIGN KEY (student_id) REFERENCES students(id),                
#                ''')
    
    # YOUR CODE HERE
    
    # Commit all changes
    conn.commit()

# Call the create_database() function
create_database(conn)

##### View your database schema:
Now, you should be able to view the sqlite3 database which you had created in your assignment directory. 
You can now play around with this database to understand the different aspects of the database you just created!!!! 

Connect to a sqlite3 database via terminal with the following command:

```
$ sqlite3 student.db
```

To view the schema of your database, you can run the following command:

~~~~
.schema
~~~~

If you need help on other meta commands, you can run 
~~~~
.help
~~~~

Finally, to exit from a sqlite3 database, you can run the following command:

~~~~
.exit
~~~~

##### Populate/update database:

** TASK 2 **: Complete the following `populate_database()` function to populate the database we just created.  Be sure to use the sqlite3 interpreter, or write some additional python commands to check your work.

In [ ]:
import sqlite3
import csv
import pandas

def populate_database(conn):
    '''
    Populates database created via populate_data() by using
    student.csv and instructor.csv
    '''
    
    c = conn.cursor()
    
    # Load student dimension table
    
    # Create a CSV reader
    reader = csv.reader(open('students.csv'))

    # skip header
    next(reader, None)
    for row in reader:
        # Convert row to lower case
        row = [item.lower() for item in row]
        try:
            c.execute('''
                INSERT INTO students (student,age,degree) VALUES (?, ?, ?)
                ''', (row[0], row[1], row[2]))
        
        except sqlite3.Error as e:
            print("students, insert error:", e.args[0])
    conn.commit()
    
    # Load the courses dimension table
    # YOUR CODE HERE
    conn.commit()
    
    # Create the fact table
    reader = csv.reader(open('fact_table.csv'))

    # skip header
    next(reader, None)

    for row in reader:
        row = [item.lower() for item in row]
        try:
            # This INSERT SELECT statement looks up and uses appropriate dimension ids 
            # for each row the of the fact_table.
            c.execute('''
                INSERT INTO fact_table (student_id, course_id, grade) 
                SELECT students.id, courses.id, ? FROM students, courses
                WHERE students.student = ? AND courses.name = ?
                ''', (row[2], row[0], row[1]))
        except sqlite3.Error as e:
            print("fact_table, insert error:", e.args[0])
        
    # Commit to the connection in order to save the changes 
    conn.commit()

    
conn = sqlite3.connect('student.db')
create_database(conn)
populate_database(conn)

# Admire your handy work
# Note: this code uses Pandas to pretty prints query results 
for t in['students', 'courses', 'fact_table']:
    print('%s:'%t)
    print(pandas.read_sql_query("SELECT * FROM %s"%t, conn), '\n')

### Using relational algebras

In this final part of the lab, your task is to write [relational algebra](https://en.wikipedia.org/wiki/Relational_algebra) expressions using the following tables that represent a very limited subset of the student dataset. 

In [ ]:
import pandas as pd
# you should regard this table as the students table
students = pd.read_csv('students.csv')
students

In [ ]:
# you should regard this table as the courses table
courses = pd.read_csv('courses.csv')
courses

In [ ]:
# you should regard this table as the fact table.
fact_table = pd.read_csv('fact_table.csv')
fact_table

Here is a relational algebra expression for the [Natural Join](https://en.wikipedia.org/wiki/Relational_algebra#Natural_join_.28.E2.8B.88.29) of two tables: 
<center> $table_A \bowtie table_B$ </center>
Below is result of $students \bowtie fact\_table$

In [ ]:
pd.merge(students, fact_table)

The Natural Join expressions above written was written using Latex equations.  Click on this cell to see how the Latex formatting in markdown works.  As you can see, markdown regard everything inside a pair of \$ \$s as a Latex equation. 


** TASK 3 **: Based on the above tables, write relational algebra expressions, using Latex, for the following statements:

1. Find the names of all students.
2. Find all student and instructor name pairs such that a student has taken an instructor's course.
3. Find the name of all students whose age is below 25.


Hints: 
- For more examples and more information on Relational Algebra we recommend viewing these CSCI1270 [slides](https://cs.brown.edu/courses/csci1270/static/files/slides/RelAlgebra.pdf).
- This CSCI1270 [Latex Cheet Sheet](http://cs.brown.edu/courses/csci1270/static/files/cs127_cheatsheet.pdf) contains a list symbols for writing relational algebra expressions. 



# YOUR CODE HERE